# Coursera Data Science Capstone
## Week 4 Peer-graded Assignment

# === Toronto Neighborhood Clustering ===

### <font color='red'> Disclaimer: Dear reader, as suggested by the instructions, all three assignments are within the same notebook. They are labeled as Part 1-3. </font>

##### Hendrik Welsch, January 2021

## == Part 1 ==

### Aim: Generating data frame with postal codes of the Toronto area

#### Import necessary packages

In [1]:
import pandas as pd # importing pandas for data frame work

#### Read all possible tables from the website into a list of data frames using pandas.read_html:

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" # website containing postal codes
dfs = pd.read_html(url) # scrape websites for tables, save them as dataframes to list

#### Lets check how many data frames were saved:

In [3]:
len(dfs)

3

#### Lets see how the data frames look:

In [4]:
for i in dfs:
    print(i.shape, i.head(3),"\n\n")

(180, 3)   Postal Code       Borough Neighbourhood
0         M1A  Not assigned  Not assigned
1         M2A  Not assigned  Not assigned
2         M3A    North York     Parkwoods 


(4, 18)                                                   0   \
0                                                NaN   
1  NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...   
2                                                 NL   

                                                  1   \
0                              Canadian postal codes   
1  NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...   
2                                                 NS   

                                                  2    3    4    5    6    7   \
0                                                NaN  NaN  NaN  NaN  NaN  NaN   
1  NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...  NaN  NaN  NaN  NaN  NaN   
2                                                 PE   NB   QC   QC   QC   ON   

    8    9    10   11   12   13   14 

#### The first data frame seems to be the postal code list, while the other two are other parts of the website that pandas took for tables. 
#### We will ignore those and focus on the first data frame. Lets save it to a new name:

In [5]:
df_pc = dfs[0] # saving df at position 0
df_pc.head(11)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


#### Now we need to clean the data frame from rows with Boroughs not assigned:

In [6]:
df_pc_clean = df_pc[df_pc.Borough != "Not assigned"] # saving df withouth not assigned postal codes as new df
df_pc_clean.head(11)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


#### Lets reset the index to start at 0:

In [7]:
df_pc_clean.reset_index(inplace=True, drop=True)
df_pc_clean.head(11)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


#### As per instructions, lets look at the shape of the df:

In [8]:
df_pc_clean.shape

(103, 3)

## == Part 2 ==

### Aim: Adding coordinates to the data frame

#### <font color='red'> Disclaimer: The geocoder approach did not work for me (always returned "None"). Thus, we will use the provided csv file to add the coordinates to our dataframe </font>

#### Lets install and import wget

In [9]:
!pip install wget
import wget

#### Next we download and save the provided csv file

In [10]:
geodata = wget.download("http://cocl.us/Geospatial_data")

100% [................................................................................] 2891 / 2891

#### Now we read the downloaded csv into a data frame:

In [11]:
df_geo = pd.read_csv(geodata)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
df_geo.shape

(103, 3)

#### Now we need to join both data frames. This will be done based on the Postal Code, which is why we set Postal Code as the new index. Then we use the inner join approach:

In [13]:
# set index for both df to Postal Code, then join dfs
df_pc_coord = pd.concat([df_pc_clean.set_index("Postal Code"), df_geo.set_index("Postal Code")], axis=1, join="inner")
df_pc_coord.head()

,Borough,Neighbourhood,Latitude,Longitude
Postal Code,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [14]:
df_pc_coord.shape

(103, 4)

#### Now that the join is complete, we reset the index again to a number.

In [15]:
df_pc_coord.reset_index(inplace=True)
df_pc_coord.head(11)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


#### This is the complete data frame!

#### <font color='blue'>Lets quickly check if the join was correct by comparing the coordinates of a postal code with one listed in the example pichture in the instructions:<font>

In [16]:
df_pc_coord.loc[df_pc_coord["Postal Code"] == "M4G"]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
23,M4G,East York,Leaside,43.70906,-79.363452


#### This looks to be correct. Perfect, now we can move on to part 3!

## == Part 3 ==

#### Importing necessary packages:

In [17]:
import numpy as np # library to handle data in a vectorized manner

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#### Installing and importing folium for maps

In [18]:
!conda install -c conda-forge folium=0.5.0 --yes 

Solving environment: ...working... done

# All requested packages already installed.



In [19]:
import folium # map rendering library

#### Importing geopy for determining coordinates

In [20]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: ...working... done

# All requested packages already installed.



#### We use geopy to determine coordinates of Toronto

In [21]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Using the coordinates, we generate a map of toronto with folium. Then we add all areas based on their postal codes as points of interest
#### Please note: Here, one has several options on how to separate the neighborhoods. I chose to use the postal codes. 

In [22]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for pc, lat, lng, borough, neighborhood in zip(df_pc_coord['Postal Code'], df_pc_coord['Latitude'], df_pc_coord['Longitude'], df_pc_coord['Borough'], df_pc_coord['Neighbourhood']):
    label = '{} ({}, {})'.format(neighborhood, borough, pc)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### To use the Foursquare API, I have to enter my credentials. For security reasons, the cell containing this data is hidden

In [23]:
# @hidden_cell
CLIENT_ID = 'ULMKGONKAZW5C2KFMPGQOK1AVGPSR3HTCUEBLKZTXCB1IRP1' # your Foursquare ID
CLIENT_SECRET = 'JFUL4Y3ISFESYFNYQK3PUZIKXLJ24ZGT1BON4RE2SK30LDK3' # your Foursquare Secret

#### Then we add the version number, a limit for search queries and a radius to search around the POIs

In [27]:
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
RADIUS = 500

#### The following function pulls the top 100 venues around a point of interest (postal code area) and saves it to a dataframe

In [28]:
def getNearbyVenues(pcs, names, latitudes, longitudes, radius):
    
    venues_list=[]
    for pc, name, lat, lng in zip(pcs, names, latitudes, longitudes):
        #print(pc, name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            pc,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ["Postal Code", 'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now we run the above function with the toronto data as input

In [31]:
toronto_venues = getNearbyVenues(
    pcs=df_pc_coord["Postal Code"],
    names=df_pc_coord["Neighbourhood"],
    latitudes=df_pc_coord["Latitude"],
    longitudes=df_pc_coord["Longitude"], 
    radius=RADIUS)

M3A Parkwoods
M4A Victoria Village
M5A Regent Park, Harbourfront
M6A Lawrence Manor, Lawrence Heights
M7A Queen's Park, Ontario Provincial Government
M9A Islington Avenue, Humber Valley Village
M1B Malvern, Rouge
M3B Don Mills
M4B Parkview Hill, Woodbine Gardens
M5B Garden District, Ryerson
M6B Glencairn
M9B West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
M1C Rouge Hill, Port Union, Highland Creek
M3C Don Mills
M4C Woodbine Heights
M5C St. James Town
M6C Humewood-Cedarvale
M9C Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
M1E Guildwood, Morningside, West Hill
M4E The Beaches
M5E Berczy Park
M6E Caledonia-Fairbanks
M1G Woburn
M4G Leaside
M5G Central Bay Street
M6G Christie
M1H Cedarbrae
M2H Hillcrest Village
M3H Bathurst Manor, Wilson Heights, Downsview North
M4H Thorncliffe Park
M5H Richmond, Adelaide, King
M6H Dufferin, Dovercourt Village
M1J Scarborough Village
M2J Fairview, Henry Farm, Oriole
M3J Northwood Park, York University
M4J East Toronto

In [32]:
toronto_venues.head()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,M4A,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,M4A,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,M4A,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


#### Lets have a look at how many unique venue categories there are in Toronto:

In [34]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 268 uniques categories.


#### Using one hot encoding, we generate an data frame with all collected venue categories as column labels

In [36]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Postal Code'] = toronto_venues['Postal Code'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Postal Code,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M3A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [37]:
toronto_onehot.shape

(2117, 269)

#### Now we generate a new data frame containing grouped data

In [38]:
toronto_grouped = toronto_onehot.groupby('Postal Code').mean().reset_index()
toronto_grouped.head()

,Postal Code,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0

In [39]:
toronto_grouped.shape

(99, 269)

#### ...note that we have only 99 Postal Codes, where it was 103 in the beginning. This is because for some coordinates, the foursquare search did not come up with any venues. This might change if  the search radius is increased (currently = 500), however, for this project we will leave it at 500
#### Now we will look at the top 5 venues per location:

In [42]:
num_top_venues = 5

for hood in toronto_grouped['Postal Code']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Postal Code'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M1B----
                      venue  freq
0      Fast Food Restaurant   1.0
1             Movie Theater   0.0
2            Massage Studio   0.0
3            Medical Center   0.0
4  Mediterranean Restaurant   0.0


----M1C----
                        venue  freq
0                         Bar  0.33
1  Construction & Landscaping  0.33
2                Home Service  0.33
3           Accessories Store  0.00
4          Miscellaneous Shop  0.00


----M1E----
                venue  freq
0      Breakfast Spot  0.12
1      Medical Center  0.12
2  Mexican Restaurant  0.12
3        Intersection  0.12
4   Electronics Store  0.12


----M1G----
                   venue  freq
0            Coffee Shop  0.67
1  Korean BBQ Restaurant  0.33
2      Accessories Store  0.00
3     Miscellaneous Shop  0.00
4    Moroccan Restaurant  0.00


----M1H----
                  venue  freq
0                Bakery  0.12
1    Athletics & Sports  0.12
2           Gas Station  0.12
3       Thai Restaurant  0.12
4  Carib

                venue  freq
0         Coffee Shop  0.13
1         Pizza Place  0.07
2                 Pub  0.07
3        Liquor Store  0.07
4  Light Rail Station  0.07


----M4W----
                       venue  freq
0                       Park  0.50
1                 Playground  0.25
2                      Trail  0.25
3          Accessories Store  0.00
4  Middle Eastern Restaurant  0.00


----M4X----
         venue  freq
0  Pizza Place  0.07
1   Restaurant  0.07
2  Coffee Shop  0.07
3    Pet Store  0.04
4         Café  0.04


----M4Y----
                 venue  freq
0          Coffee Shop  0.08
1     Sushi Restaurant  0.06
2  Japanese Restaurant  0.06
3              Gay Bar  0.04
4           Restaurant  0.04


----M5A----
         venue  freq
0  Coffee Shop  0.15
1         Park  0.07
2         Café  0.07
3       Bakery  0.07
4          Pub  0.04


----M5B----
                       venue  freq
0             Clothing Store  0.09
1                Coffee Shop  0.09
2             Cosmeti

                      venue  freq
0            Sandwich Place   1.0
1         Accessories Store   0.0
2            Massage Studio   0.0
3            Medical Center   0.0
4  Mediterranean Restaurant   0.0


----M9V----
                 venue  freq
0        Grocery Store  0.25
1             Pharmacy  0.12
2          Pizza Place  0.12
3           Beer Store  0.12
4  Fried Chicken Joint  0.12


----M9W----
                 venue  freq
0           Truck Stop   0.2
1                  Bar   0.2
2  Rental Car Location   0.2
3            Drugstore   0.2
4        Garden Center   0.2




#### Next, we will define a function to return the top venues categories per location and run it for every location. The result is a data frame.

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [107]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postal Code'] = toronto_grouped['Postal Code']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio,Dessert Shop
1,M1C,Bar,Construction & Landscaping,Home Service,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
2,M1E,Bank,Breakfast Spot,Electronics Store,Medical Center,Restaurant,Rental Car Location,Intersection,Mexican Restaurant,Yoga Studio,Distribution Center
3,M1G,Coffee Shop,Korean BBQ Restaurant,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
4,M1H,Bakery,Hakka Restaurant,Caribbean Restaurant,Fried Chicken Joint,Thai Restaurant,Gas Station,Athletics & Sports,Bank,Distribution Center,Dim Sum Restaurant


#### To cluster the neighborhoods, we generate a k means model and label the locations based on their cluster

In [108]:
# set number of clusters
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Postal Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 1, 3, 3, 3, 3])

In [109]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_pc_coord

In [110]:
# merge toronto_grouped with neighborhoods_venues_sorted to add latitude/longitude for each postal code area
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Postal Code'), on='Postal Code')

toronto_merged.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,5.0,Food & Drink Shop,Park,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
1,M4A,North York,Victoria Village,43.725882,-79.315572,3.0,French Restaurant,Coffee Shop,Portuguese Restaurant,Hockey Arena,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3.0,Coffee Shop,Park,Bakery,Café,Breakfast Spot,Pub,Theater,Yoga Studio,Brewery,Shoe Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,3.0,Clothing Store,Accessories Store,Vietnamese Restaurant,Boutique,Furniture / Home Store,Coffee Shop,Event Space,Construction & Landscaping,Convenience Store,Comfort Food Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,3.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Smoothie Shop,Burrito Place,Sandwich Place,Café,Portuguese Restaurant,College Auditorium,Park
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,3.0,Fast Food Restaurant,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio,Dessert Shop
7,M3B,North York,Don Mills,43.745906,-79.352188,3.0,Japanese Restaurant,Gym,Caribbean Restaurant,Café,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,3.0,Pizza Place,Pharmacy,Gastropub,Flea Market,Bank,Intersection,Athletics & Sports,Breakfast Spot,Pet Store,Gym / Fitness Center
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3.0,Coffee Shop,Clothing Store,Bubble Tea Shop,Japanese Restaurant,Cosmetics Shop,Middle Eastern Restaurant,Hotel,Café,Ramen Restaurant,Theater


#### As mentioned earlier, there are some areas without any venues (e.g. M9A), which only contain NaN in the table.

In [133]:
toronto_merged["Cluster Labels"].unique()

array([5., 3., 1., 2., 0., 4.])

#### Lets look at the data that we can retreive from Foursquare for postal code M9A

In [134]:
df_pc_coord.loc[df_pc_coord["Postal Code"] == "M9A"] # show data frame with coordinates

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242


In [135]:
# url with coordinates for M9A
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            43.667856, 
            79.532242, 
            RADIUS, 
            LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ffb648ae53c096fdf78925f'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 0,
  'suggestedBounds': {'ne': {'lat': 43.6723560045, 'lng': 79.53845140585746},
   'sw': {'lat': 43.6633559955, 'lng': 79.52603259414254}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': []}]}}

#### Note the warning:
'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."

#### Lets drop the rows with NaN:

In [112]:
toronto_merged.dropna(subset=["Cluster Labels"], inplace=True)
toronto_merged.shape

(99, 16)

In [113]:
toronto_merged["Cluster Labels"].unique()

array([5., 3., 1., 2., 0., 4.])

#### ... now there are only clustered areas left in the data frame!
#### We can now use folium to add the locations to a map of Toronto, where the locations is colored based on its cluster:

In [144]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
alt_colors = ["black", "green", "blue", "red", "purple", "orange"]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, pc, bor in zip(
    toronto_merged['Latitude'], 
    toronto_merged['Longitude'], 
    toronto_merged['Neighbourhood'], 
    toronto_merged['Cluster Labels'], 
    toronto_merged['Postal Code'], 
    toronto_merged['Borough']):
    
    label = folium.Popup(str(pc)+ ': ' + str(poi) + ' (Cluster ' + str(int(cluster)) + ')', parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=alt_colors[int(cluster)-1],
        fill=True,
        fill_color=alt_colors[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Based on k=6 and radius of 500 m, we can describe the cluster as follows:

#### Cluster 0 (orange): Locations in this cluster are defined by their vincinity to baseball fields.

In [119]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,North York,0.0,Food Truck,Baseball Field,Home Service,Yoga Studio,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
57,North York,0.0,Construction & Landscaping,Baseball Field,Yoga Studio,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Electronics Store
101,Etobicoke,0.0,Baseball Field,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Field


#### Cluster 1 (black): Locations in this cluster are defined by an increased number of pizza and sandwhich places. They are enriched in the north west part of Toronto.

In [121]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,North York,1.0,Pizza Place,Japanese Restaurant,Bakery,Pub,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dessert Shop
32,Scarborough,1.0,Pizza Place,Playground,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
50,North York,1.0,Pizza Place,Furniture / Home Store,Intersection,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Farmers Market,Doner Restaurant
56,York,1.0,Sandwich Place,Discount Store,Bar,Skating Rink,Yoga Studio,Donut Shop,Distribution Center,Dog Run,Doner Restaurant,Drugstore
70,Etobicoke,1.0,Pizza Place,Coffee Shop,Sandwich Place,Discount Store,Chinese Restaurant,Intersection,Eastern European Restaurant,Drugstore,Donut Shop,Department Store
71,Scarborough,1.0,Bakery,Auto Garage,Smoke Shop,Sandwich Place,Middle Eastern Restaurant,Yoga Studio,Dog Run,Discount Store,Distribution Center,Doner Restaurant
77,Etobicoke,1.0,Sandwich Place,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
89,Etobicoke,1.0,Grocery Store,Pizza Place,Fried Chicken Joint,Sandwich Place,Fast Food Restaurant,Beer Store,Pharmacy,Gift Shop,Dance Studio,Eastern European Restaurant
93,Etobicoke,1.0,Pizza Place,Sandwich Place,Gym,Coffee Shop,Skating Rink,Pool,Pub,Yoga Studio,Distribution Center,Dessert Shop


#### Cluster 2 (green): There is only one location in this cluster. Here, the presence of Filipino restaurant(s) seems to be the defining factor.

In [122]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,2.0,Filipino Restaurant,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant


#### Cluster 3 (blue): Most locations group into cluster 3. Here, coffee shops, various restaurants, shops and bars define the area. Since locations in this cluster are spread all over Toronto, one can assume that these areas are normal living/shopping areas.

In [123]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,3.0,French Restaurant,Coffee Shop,Portuguese Restaurant,Hockey Arena,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
2,Downtown Toronto,3.0,Coffee Shop,Park,Bakery,Café,Breakfast Spot,Pub,Theater,Yoga Studio,Brewery,Shoe Store
3,North York,3.0,Clothing Store,Accessories Store,Vietnamese Restaurant,Boutique,Furniture / Home Store,Coffee Shop,Event Space,Construction & Landscaping,Convenience Store,Comfort Food Restaurant
4,Downtown Toronto,3.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Smoothie Shop,Burrito Place,Sandwich Place,Café,Portuguese Restaurant,College Auditorium,Park
6,Scarborough,3.0,Fast Food Restaurant,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio,Dessert Shop
7,North York,3.0,Japanese Restaurant,Gym,Caribbean Restaurant,Café,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
8,East York,3.0,Pizza Place,Pharmacy,Gastropub,Flea Market,Bank,Intersection,Athletics & Sports,Breakfast Spot,Pet Store,Gym / Fitness Center
9,Downtown Toronto,3.0,Coffee Shop,Clothing Store,Bubble Tea Shop,Japanese Restaurant,Cosmetics Shop,Middle Eastern Restaurant,Hotel,Café,Ramen Restaurant,Theater
12,Scarborough,3.0,Bar,Construction & Landscaping,Home Service,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
13,North York,3.0,Restaurant,Gym,Coffee Shop,Beer Store,Dim Sum Restaurant,Japanese Restaurant,Discount Store,Supermarket,Sandwich Place,Chinese Restaurant


#### Cluster 4 (red): As with cluster 2, there is only one location in this cluster. It seems to be defined by a summer camp.

In [124]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Central Toronto,4.0,Summer Camp,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant


#### Cluster 5 (purple): The last cluster is defined by parks and playgrounds.

In [126]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,5.0,Food & Drink Shop,Park,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
21,York,5.0,Park,Women's Store,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
35,East York,5.0,Park,Convenience Store,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
40,North York,5.0,Airport,Park,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
64,York,5.0,Park,Jewelry Store,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
66,North York,5.0,Convenience Store,Park,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
85,Scarborough,5.0,Playground,Park,Intersection,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
91,Downtown Toronto,5.0,Park,Playground,Trail,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run


## Thank you for taking your time and reviewing my work!